In [1]:
NEO4J_URI="neo4j+s://c0fa349b.databases.neo4j.io"
NEO4J_USERNAME="neo4j"
NEO4J_PASSWORD="vd22KIBa2I9OP7ElQP3U0kGGQ-X_bljNsjswOutijts"

In [2]:
import os
os.environ['NEO4J_URI']=NEO4J_URI
os.environ['NEO4J_USERNAME']=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [3]:
from langchain_community.graphs import Neo4jGraph
graph=Neo4jGraph(url=NEO4J_URI,username=NEO4J_USERNAME,password=NEO4J_PASSWORD)
graph

/Users/aditya/PycharmProjects/LLM/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [4]:
## Dataset
movie_query="""
LOAD CSV WITH HEADERS FROM 
'https://raw.githubusercontent.com/tomasonjo/blog-datasets/refs/heads/main/movies/movies_small.csv' AS row

MERGE (m:Movie {id: row.movieId})
SET m.released = row.released,
    m.title = row.title,
    m.imdbRating = toFloat(row.imdbRating)

FOREACH (director IN split(row.director, '|') |
    MERGE (d:Person {name: trim(director)})
    MERGE (d)-[:DIRECTED]->(m)
)

FOREACH (actor IN split(row.actors, '|') |
    MERGE (a:Person {name: trim(actor)})
    MERGE (a)-[:ACTED_IN]->(m)
)

FOREACH (genre IN split(row.genres, '|') |
    MERGE (g:Genre {name: trim(genre)})
    MERGE (m)-[:IN_GENRE]->(g)
)

"""

In [5]:
movie_query

"\nLOAD CSV WITH HEADERS FROM \n'https://raw.githubusercontent.com/tomasonjo/blog-datasets/refs/heads/main/movies/movies_small.csv' AS row\n\nMERGE (m:Movie {id: row.movieId})\nSET m.released = row.released,\n    m.title = row.title,\n    m.imdbRating = toFloat(row.imdbRating)\n\nFOREACH (director IN split(row.director, '|') |\n    MERGE (d:Person {name: trim(director)})\n    MERGE (d)-[:DIRECTED]->(m)\n)\n\nFOREACH (actor IN split(row.actors, '|') |\n    MERGE (a:Person {name: trim(actor)})\n    MERGE (a)-[:ACTED_IN]->(m)\n)\n\nFOREACH (genre IN split(row.genres, '|') |\n    MERGE (g:Genre {name: trim(genre)})\n    MERGE (m)-[:IN_GENRE]->(g)\n)\n\n"

In [6]:
graph.query(movie_query)

[]

In [7]:
graph.refresh_schema()
print(graph.schema)

Node properties:
Movie {id: STRING, released: STRING, title: STRING, imdbRating: FLOAT}
Person {name: STRING}
Genre {name: STRING}
Relationship properties:

The relationships:
(:Movie)-[:IN_GENRE]->(:Genre)
(:Person)-[:DIRECTED]->(:Movie)
(:Person)-[:ACTED_IN]->(:Movie)


In [9]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")

In [10]:
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Gemma2-9b-It")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x13568d610>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x135681730>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))

In [11]:
from langchain.chains import GraphCypherQAChain
chain=GraphCypherQAChain.from_llm(graph=graph,llm=llm,verbose=True)
chain

GraphCypherQAChain(verbose=True, graph=<langchain_community.graphs.neo4j_graph.Neo4jGraph object at 0x105dce1c0>, cypher_generation_chain=LLMChain(prompt=PromptTemplate(input_variables=['question', 'schema'], template='Task:Generate Cypher statement to query a graph database.\nInstructions:\nUse only the provided relationship types and properties in the schema.\nDo not use any other relationship types or properties that are not provided.\nSchema:\n{schema}\nNote: Do not include any explanations or apologies in your responses.\nDo not respond to any questions that might ask anything else than for you to construct a Cypher statement.\nDo not include any text except the generated Cypher statement.\n\nThe question is:\n{question}'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x13568d610>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x135681730>, model_name='Gemma2-9b-It', groq_api_key=SecretStr('**********'))), qa_chain=LLMChain(

In [13]:
response=chain.invoke({"query":"Who was the director of the movie Heat"})
response



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: 'Heat'})<-[:DIRECTED]-(p:Person) RETURN p.name 

Full Context:
[{'p.name': 'Michael Mann'}]

> Finished chain.


{'query': 'Who was the director of the movie Heat',
 'result': 'Michael Mann \n'}